# Analisis Sentimen Ulasan Aplikasi EdLink di Google Play Store Menggunakan Metode Klasifikasi Algoritma Naive Bayes

#Scraping Data

In [1]:
#Install Google Play Store Scrapper
!pip install google-play-scraper

#Install Stemming
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.4 MB/s eta 0:00:00


#Import Paket Yang Diperlukan

In [2]:
from google_play_scraper import app
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews
import re
import nltk.corpus
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
#Scrape Jumlah Ulasan Yang Diinginkan
result, continuation_token = reviews(
    'id.co.sevima.edlink', #Disini kita mau men-scrape data ulasan aplikasi EdLink yang berada di google play store
    lang='id',  #Kita setting bahasanya menjadi bahasa Indonesia
    country='id', #Kita setting negaranya menjadi negara Indonesia
    sort=Sort.MOST_RELEVANT, #Kemudian kita gunakan most_relevan untuk mendapatkan ulasan yang paling relevant
    count=1000, #Disini jumlah ulasan yang mau kita ambil ada seribu
    filter_score_with=None #Kemudian di filter_score kita gunakan None untuk mengambil semua score atau rating bintang 1 sampai 5
)

In [4]:
df_busu = pd.DataFrame(np.array(result),columns=['review'])
df_busu = df_busu.join(pd.DataFrame(df_busu.pop('review').tolist()))
df_busu.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,f0ed341e-cad1-4e48-8b62-6e2363b6538c,Ayu marcellah,https://play-lh.googleusercontent.com/a/ACg8oc...,"Akun tiba-tiba keluar sendiri, kalau login ula...",1,18,4.7.9,2023-11-24 01:45:14,"Halo Kak Ayu, Silakan re-login ke Edlink dan l...",2023-11-25 12:35:36,4.7.9
1,e13afe8c-848a-496d-959e-66521252883d,Nadia Rahma M,https://play-lh.googleusercontent.com/a-/ALV-U...,"Menurut saya, apknya udah bagus dan membantu.....",4,4,4.7.9,2023-11-30 01:41:03,Halo Kak Nadia. Terima kasih atas ulasannya. F...,2023-12-04 08:06:15,4.7.9
2,de48db13-506b-45e0-963d-8f8bf9ed5b93,OmsahNza Official,https://play-lh.googleusercontent.com/a-/ALV-U...,Kenapa kalo pas ujian yg gak buka tab lain yan...,1,13,4.7.9,2023-12-10 05:53:40,None,NaT,4.7.9
3,e1f96c10-ca28-46e0-960f-47e47b48e9d5,Tara Alamsyah,https://play-lh.googleusercontent.com/a-/ALV-U...,Kepada developer mohon untuk mengoptimalisasi ...,1,0,4.8.0,2024-01-12 09:19:21,None,NaT,4.8.0
4,be6cccbb-9f2e-4f65-bb4e-fa1dd4da8d1d,Topik Nurmansyah,https://play-lh.googleusercontent.com/a-/ALV-U...,"Scan absennya gak bisa di zoom, jadi kepaksa h...",1,1,4.7.9,2023-12-04 15:48:45,None,NaT,4.7.9


In [5]:
len(df_busu.index) #Kemudian hitung kembali berapa jumlah data yg didapatkan

1000

In [6]:
df_busu[['userName', 'score','at', 'content']].head()  #Dari scrapping tsb didapatkan banyak sekali kolom, kemudian kolom" tsb kita filter
                                                       #Sehingga didapatkan kolom username, score, at dan content

,userName,score,at,content
0,Ayu marcellah,1,2023-11-24 01:45:14,"Akun tiba-tiba keluar sendiri, kalau login ula..."
1,Nadia Rahma M,4,2023-11-30 01:41:03,"Menurut saya, apknya udah bagus dan membantu....."
2,OmsahNza Official,1,2023-12-10 05:53:40,Kenapa kalo pas ujian yg gak buka tab lain yan...
3,Tara Alamsyah,1,2024-01-12 09:19:21,Kepada developer mohon untuk mengoptimalisasi ...
4,Topik Nurmansyah,1,2023-12-04 15:48:45,"Scan absennya gak bisa di zoom, jadi kepaksa h..."


In [7]:
#Mengurutkan data berdasarkan tanggal
new_df = df_busu[['userName', 'score','at', 'content']]
sorted_df = new_df.sort_values(by='at', ascending=False) #Diurutkan dari yang terbaru, ubah menjadi True untuk melihat dari yang terdahulu.
sorted_df.head()

,userName,score,at,content
50,hanifa indriani,2,2024-01-17 07:53:06,"Notif gak masuk sama sekali, errorr."
82,sun shine,3,2024-01-17 02:58:53,masih bingung dengan tagihan mahasiswa yang ti...
30,Allya Ameliana,1,2024-01-12 13:28:34,"Aplikasi ini sumpah jelek banget bikin ribet, ..."
3,Tara Alamsyah,1,2024-01-12 09:19:21,Kepada developer mohon untuk mengoptimalisasi ...
48,Rafli Hafidli,1,2024-01-02 06:12:38,Scan barcode presentasi selalu Ga Valid


In [8]:
my_df = sorted_df[['content', 'score']] #Karena kita hanya membutuhkan kolom content dan score maka kita lakukan filter kolom lgi hingga menyisakan kolom content dan score.


In [9]:
my_df.head()

,content,score
50,"Notif gak masuk sama sekali, errorr.",2
82,masih bingung dengan tagihan mahasiswa yang ti...,3
30,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1
3,Kepada developer mohon untuk mengoptimalisasi ...,1
48,Scan barcode presentasi selalu Ga Valid,1


# PELABELAN

In [10]:
def pelabelan(score):
  if score < 3:
    return 'Negatif'
  elif score == 4 :
    return 'Positif'
  elif score == 5 :
    return 'Positif'
my_df['Label'] = my_df ['score'].apply(pelabelan)
my_df.head(50)

<ipython-input-10-310ae01c7488>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['Label'] = my_df ['score'].apply(pelabelan)


,content,score,Label
50,"Notif gak masuk sama sekali, errorr.",2,Negatif
82,masih bingung dengan tagihan mahasiswa yang ti...,3,None
30,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif
3,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif
48,Scan barcode presentasi selalu Ga Valid,1,Negatif
126,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif
32,"Tolong kepada pihak apk sevima, akun saya saat...",2,Negatif
5,aneh kenapa tiba tiba keluar sendiri dari akun...,1,Negatif
31,"Suruh kasih izin mulu, padahal udah dikasih iz...",1,Negatif
458,Halo mengapa device saya tidak bisa menerima t...,3,None


In [11]:
my_df.to_csv("scrapped_data.csv", index = False)  #kemudian save menjadi file csv

#Data Cleaning

In [12]:
pd.set_option('display.max_columns', None)
my_df = pd.read_csv('/content/scrapped_data.csv')
my_df.head(50)

,content,score,Label
0,"Notif gak masuk sama sekali, errorr.",2,Negatif
1,masih bingung dengan tagihan mahasiswa yang ti...,3,NaN
2,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif
3,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif
4,Scan barcode presentasi selalu Ga Valid,1,Negatif
5,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif
6,"Tolong kepada pihak apk sevima, akun saya saat...",2,Negatif
7,aneh kenapa tiba tiba keluar sendiri dari akun...,1,Negatif
8,"Suruh kasih izin mulu, padahal udah dikasih iz...",1,Negatif
9,Halo mengapa device saya tidak bisa menerima t...,3,NaN


In [13]:
#Info() digunakan untuk menampilkan informasi detail tentang dataframe,
#Seperti jumlah baris data, nama-nama kolom berserta jumlah data dan tipe datanya, dan sebagainya.
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  1000 non-null   object
 1   score    1000 non-null   int64 
 2   Label    886 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [14]:
#Tampilkan setiap baris yang memiliki nilai null (NaN) pada kolom apapun
#Gunakan fitur isna() yang disediakan library pandas
my_df.isna()

,content,score,Label
0,False,False,False
1,False,False,True
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
995,False,False,False
996,False,False,False
997,False,False,False
998,False,False,True


In [15]:
my_df.isna().any()

content    False
score      False
Label       True
dtype: bool

In [16]:
my_df.describe()

,score
count,1000.000000
mean,2.653000
std,1.658717
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,5.000000


In [17]:
#Mencari jumlah baris data yang bernilai null
#Terdapat kolom label memiliki nilai kosong
my_df.isnull().sum()

content      0
score        0
Label      114
dtype: int64

# 1. Handling Missing value-Ignore tuple


In [18]:
my_df.dropna(subset=['Label'],inplace = True)

In [19]:
my_df.isnull().sum()

content    0
score      0
Label      0
dtype: int64

In [20]:
my_df.head(50)

,content,score,Label
0,"Notif gak masuk sama sekali, errorr.",2,Negatif
2,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif
3,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif
4,Scan barcode presentasi selalu Ga Valid,1,Negatif
5,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif
6,"Tolong kepada pihak apk sevima, akun saya saat...",2,Negatif
7,aneh kenapa tiba tiba keluar sendiri dari akun...,1,Negatif
8,"Suruh kasih izin mulu, padahal udah dikasih iz...",1,Negatif
11,"Kenapa kodenya tidak muncul"" di Gmail yang di ...",1,Negatif
12,"Saya tidak bisa login, ketitka login malah dis...",1,Negatif


In [21]:
my_df.to_csv("edlinkpreprocessing.csv", index = False)  #Simpan hasil file data cleaning dengan nama edlinkpreprocessing.csv

# Text PreProcessing

In [22]:
df = pd.read_csv('/content/edlinkpreprocessing.csv')
df.head(50)

,content,score,Label
0,"Notif gak masuk sama sekali, errorr.",2,Negatif
1,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif
2,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif
3,Scan barcode presentasi selalu Ga Valid,1,Negatif
4,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif
5,"Tolong kepada pihak apk sevima, akun saya saat...",2,Negatif
6,aneh kenapa tiba tiba keluar sendiri dari akun...,1,Negatif
7,"Suruh kasih izin mulu, padahal udah dikasih iz...",1,Negatif
8,"Kenapa kodenya tidak muncul"" di Gmail yang di ...",1,Negatif
9,"Saya tidak bisa login, ketitka login malah dis...",1,Negatif


# Case Folding
Proses case folding adalah proses mengubah seluruh huruf menjadi huruf kecil. Pada proses ini karakter-karakter 'A'-'Z' yang terdapat pada data diubah kedalam karakter 'a'-'z'.

In [23]:
def  clean_text(df, text_field, new_text_field_name):
    my_df[new_text_field_name] = my_df[text_field].str.lower()
    my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    #Remove numbers
    my_df[new_text_field_name] = my_df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    return my_df

In [24]:
my_df['text_clean'] = my_df['content'].str.lower()
my_df['text_clean']
data_clean = clean_text(my_df, 'content', 'text_clean')
data_clean.head(10)

,content,score,Label,text_clean
0,"Notif gak masuk sama sekali, errorr.",2,Negatif,notif gak masuk sama sekali errorr
2,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif,aplikasi ini sumpah jelek banget bikin ribet k...
3,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif,kepada developer mohon untuk mengoptimalisasi ...
4,Scan barcode presentasi selalu Ga Valid,1,Negatif,scan barcode presentasi selalu ga valid
5,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif,saya udah kumpul tugas tapi file tugas yang su...
6,"Tolong kepada pihak apk sevima, akun saya saat...",2,Negatif,tolong kepada pihak apk sevima akun saya saat ...
7,aneh kenapa tiba tiba keluar sendiri dari akun...,1,Negatif,aneh kenapa tiba tiba keluar sendiri dari akun...
8,"Suruh kasih izin mulu, padahal udah dikasih iz...",1,Negatif,suruh kasih izin mulu padahal udah dikasih izi...
11,"Kenapa kodenya tidak muncul"" di Gmail yang di ...",1,Negatif,kenapa kodenya tidak muncul di gmail yang di p...
12,"Saya tidak bisa login, ketitka login malah dis...",1,Negatif,saya tidak bisa login ketitka login malah disu...


# Stopword Removal
Stopword adalah kata umum yang biasanya muncul dalam jumlah besar dan dianggap tidak memiliki makna. Contoh stopword dalam bahasa Indonesia adalah “yang”, “dan”, “di”, “dari”, dll. Makna di balik penggunaan stopword yaitu dengan menghapus kata-kata yang memiliki informasi rendah dari sebuah teks, kita dapat fokus pada kata-kata penting sebagai gantinya.

In [25]:
nltk.download('stopwords')

stop = stopwords.words('indonesian')
data_clean['text_StopWord'] = data_clean['text_clean'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
data_clean.head(50)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,content,score,Label,text_clean,text_StopWord
0,"Notif gak masuk sama sekali, errorr.",2,Negatif,notif gak masuk sama sekali errorr,notif gak masuk errorr
2,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif,aplikasi ini sumpah jelek banget bikin ribet k...,aplikasi sumpah jelek banget bikin ribet kenda...
3,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif,kepada developer mohon untuk mengoptimalisasi ...,developer mohon mengoptimalisasi aplikasi andr...
4,Scan barcode presentasi selalu Ga Valid,1,Negatif,scan barcode presentasi selalu ga valid,scan barcode presentasi ga valid
5,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif,saya udah kumpul tugas tapi file tugas yang su...,udah kumpul tugas file tugas dikumpulkan hilan...
6,"Tolong kepada pihak apk sevima, akun saya saat...",2,Negatif,tolong kepada pihak apk sevima akun saya saat ...,tolong apk sevima akun mengakses materi dosen ...
7,aneh kenapa tiba tiba keluar sendiri dari akun...,1,Negatif,aneh kenapa tiba tiba keluar sendiri dari akun...,aneh akun masuk ga terhubung kelas perguruan m...
8,"Suruh kasih izin mulu, padahal udah dikasih iz...",1,Negatif,suruh kasih izin mulu padahal udah dikasih izi...,suruh kasih izin mulu udah dikasih izin penyim...
11,"Kenapa kodenya tidak muncul"" di Gmail yang di ...",1,Negatif,kenapa kodenya tidak muncul di gmail yang di p...,kodenya muncul gmail pakai
12,"Saya tidak bisa login, ketitka login malah dis...",1,Negatif,saya tidak bisa login ketitka login malah disu...,login ketitka login disuruh melengkapi data am...


# Tokenizing
Tokenizing adalah proses pemisahan teks menjadi potongan-potongan yang disebut sebagai token untuk kemudian di analisa. Kata, angka, simbol, tanda baca dan entitas penting lainnya dapat dianggap sebagai token. Didalam NLP, token diartikan sebagai “kata” meskipun tokenize juga dapat dilakukan pada paragraf maupun kalimat

In [26]:
nltk.download('punkt')

data_clean['text_tokens'] = data_clean['text_StopWord'].apply(lambda x: word_tokenize(x))
data_clean.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,content,score,Label,text_clean,text_StopWord,text_tokens
0,"Notif gak masuk sama sekali, errorr.",2,Negatif,notif gak masuk sama sekali errorr,notif gak masuk errorr,"[notif, gak, masuk, errorr]"
2,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif,aplikasi ini sumpah jelek banget bikin ribet k...,aplikasi sumpah jelek banget bikin ribet kenda...,"[aplikasi, sumpah, jelek, banget, bikin, ribet..."
3,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif,kepada developer mohon untuk mengoptimalisasi ...,developer mohon mengoptimalisasi aplikasi andr...,"[developer, mohon, mengoptimalisasi, aplikasi,..."
4,Scan barcode presentasi selalu Ga Valid,1,Negatif,scan barcode presentasi selalu ga valid,scan barcode presentasi ga valid,"[scan, barcode, presentasi, ga, valid]"
5,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif,saya udah kumpul tugas tapi file tugas yang su...,udah kumpul tugas file tugas dikumpulkan hilan...,"[udah, kumpul, tugas, file, tugas, dikumpulkan..."


# Stemming
Stemming adalah proses pemetaan dan penguraian bentuk dari suatu kata menjadi bentuk kata dasarnya. Untuk melakukan stemming bahasa Indonesia kita dapat menggunakan library Python Sastrawi yang sudah kita siapkan di awal. Library Sastrawi menerapkan Algoritma Nazief dan Adriani dalam melakukan stemming bahasa Indonesia.

In [27]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [28]:
#-----------------STEMMING -----------------
#Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#Stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}
hitung=0

for document in data_clean['text_tokens']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    hitung+=1
    print(hitung,":",term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

#Apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]


#Script ini bisa dipisah dari eksekusinya setelah pembacaaan term selesai
data_clean['text_steamindo'] = data_clean['text_tokens'].apply(lambda x:' '.join(get_stemmed_term(x)))
data_clean.head(20)

2394
------------------------
1 : notif : notif
2 : gak : gak
3 : masuk : masuk
4 : errorr : errorr
5 : aplikasi : aplikasi
6 : sumpah : sumpah
7 : jelek : jelek
8 : banget : banget
9 : bikin : bikin
10 : ribet : ribet
11 : kendala : kendala
12 : ga : ga
13 : layak : layak
14 : kasih : kasih
15 : bintang : bintang
16 : aja : aja
17 : developer : developer
18 : mohon : mohon
19 : mengoptimalisasi : optimalisasi
20 : android : android
21 : u : u
22 : bug : bug
23 : detail : detail
24 : kelas : kelas
25 : yg : yg
26 : diakses : akses
27 : diupdate : diupdate
28 : scan : scan
29 : barcode : barcode
30 : presentasi : presentasi
31 : valid : valid
32 : udah : udah
33 : kumpul : kumpul
34 : tugas : tugas
35 : file : file
36 : dikumpulkan : kumpul
37 : hilang : hilang
38 : gimana : gimana
39 : solusinya : solusi
40 : ya : ya
41 : kak : kak
42 : tolong : tolong
43 : apk : apk
44 : sevima : sevima
45 : akun : akun
46 : mengakses : akses
47 : materi : materi
48 : dosen : dosen
49 : twetulis : twe

,content,score,Label,text_clean,text_StopWord,text_tokens,text_steamindo
0,"Notif gak masuk sama sekali, errorr.",2,Negatif,notif gak masuk sama sekali errorr,notif gak masuk errorr,"[notif, gak, masuk, errorr]",notif gak masuk errorr
2,"Aplikasi ini sumpah jelek banget bikin ribet, ...",1,Negatif,aplikasi ini sumpah jelek banget bikin ribet k...,aplikasi sumpah jelek banget bikin ribet kenda...,"[aplikasi, sumpah, jelek, banget, bikin, ribet...",aplikasi sumpah jelek banget bikin ribet kenda...
3,Kepada developer mohon untuk mengoptimalisasi ...,1,Negatif,kepada developer mohon untuk mengoptimalisasi ...,developer mohon mengoptimalisasi aplikasi andr...,"[developer, mohon, mengoptimalisasi, aplikasi,...",developer mohon optimalisasi aplikasi android ...
4,Scan barcode presentasi selalu Ga Valid,1,Negatif,scan barcode presentasi selalu ga valid,scan barcode presentasi ga valid,"[scan, barcode, presentasi, ga, valid]",scan barcode presentasi ga valid
5,"Saya udah kumpul tugas, tapi file tugas yang s...",1,Negatif,saya udah kumpul tugas tapi file tugas yang su...,udah kumpul tugas file tugas dikumpulkan hilan...,"[udah, kumpul, tugas, file, tugas, dikumpulkan...",udah kumpul tugas file tugas kumpul hilang gim...
6,"Tolong kepada pihak apk sevima, akun saya saat...",2,Negatif,tolong kepada pihak apk sevima akun saya saat ...,tolong apk sevima akun mengakses materi dosen ...,"[tolong, apk, sevima, akun, mengakses, materi,...",tolong apk sevima akun akses materi dosen twet...
7,aneh kenapa tiba tiba keluar sendiri dari akun...,1,Negatif,aneh kenapa tiba tiba keluar sendiri dari akun...,aneh akun masuk ga terhubung kelas perguruan m...,"[aneh, akun, masuk, ga, terhubung, kelas, perg...",aneh akun masuk ga hubung kelas guru menggangu...
8,"Suruh kasih izin mulu, padahal udah dikasih iz...",1,Negatif,suruh kasih izin mulu padahal udah dikasih izi...,suruh kasih izin mulu udah dikasih izin penyim...,"[suruh, kasih, izin, mulu, udah, dikasih, izin...",suruh kasih izin mulu udah kasih izin simpan k...
11,"Kenapa kodenya tidak muncul"" di Gmail yang di ...",1,Negatif,kenapa kodenya tidak muncul di gmail yang di p...,kodenya muncul gmail pakai,"[kodenya, muncul, gmail, pakai]",kode muncul gmail pakai
12,"Saya tidak bisa login, ketitka login malah dis...",1,Negatif,saya tidak bisa login ketitka login malah disu...,login ketitka login disuruh melengkapi data am...,"[login, ketitka, login, disuruh, melengkapi, d...",login ketitka login suruh lengkap data aman ko...


In [29]:
data_clean.to_csv('hasil_TextPreProcessing_edlink.csv', index= False) #Kemudian simpan hasil text preprocessing ke file csv

In [30]:
#Kemudian kita lakukan praproses
def praproses(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)()(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

#Spliting Data

Memecah data test 20% dari keseluruhan data

In [31]:
#Membagi data menjadi data training dan testing dengan test_size = 0.2 dan random state nya 0
X_train, X_test, y_train, y_test = train_test_split(data_clean['content'], data_clean['Label'],
                                                    test_size = 0.20,
                                                    random_state = 0)

#Pembobotan tf-idf

In [32]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

In [33]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(708,)
(708,)
(178,)
(178,)


In [34]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [35]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

#Naive Bayes

In [36]:
nb = MultinomialNB()
nb.fit(tfidf_train, y_train)

MultinomialNB()

In [37]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
y_pred = nb.predict(tfidf_test)

In [39]:
accuracy = accuracy_score(y_test, y_pred)

In [40]:
#Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)

print("MultinomialNB Accuracy:", accuracy_score(y_test,predicted))
print("MultinomialNB Precision:", precision_score(y_test,predicted, average="binary", pos_label="Negatif"))
print("MultinomialNB Recall:", recall_score(y_test,predicted, average="binary", pos_label="Negatif"))
print("MultinomialNB f1_score:", f1_score(y_test,predicted, average="binary", pos_label="Negatif"))

print(f'confusion_matrix:\n {confusion_matrix(y_test, predicted)}')
print('====================================================\n')
print(classification_report(y_test, predicted, zero_division=0))

#Load dataset
data_clean = pd.read_csv('hasil_TextPreProcessing_edlink.csv')

MultinomialNB Accuracy: 0.848314606741573
MultinomialNB Precision: 0.8292682926829268
MultinomialNB Recall: 0.9444444444444444
MultinomialNB f1_score: 0.883116883116883
confusion_matrix:
 [[102   6]
 [ 21  49]]

              precision    recall  f1-score   support

     Negatif       0.83      0.94      0.88       108
     Positif       0.89      0.70      0.78        70

    accuracy                           0.85       178
   macro avg       0.86      0.82      0.83       178
weighted avg       0.85      0.85      0.84       178



#Support Vector Machine

In [41]:
svmm = svm.SVC()
svmm.fit(tfidf_train, y_train)

SVC()

In [42]:
X_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [43]:
y_pred = svmm.predict(tfidf_test)

In [44]:
accuracy = accuracy_score(y_test, y_pred)

In [45]:
#Support Vector Machine
clf = svm.SVC()
clf.fit(X_train, y_train)
predicted = clf.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, predicted))
print("SVM Precision:", precision_score(y_test, predicted, average="binary", pos_label="Negatif"))
print("SVM Recall:", recall_score(y_test, predicted, average="binary", pos_label="Negatif"))
print("SVM f1_score:", f1_score(y_test, predicted, average="binary", pos_label="Negatif"))

print(f'confusion_matrix:\n {confusion_matrix(y_test, predicted)}')
print('====================================================\n')
print(classification_report(y_test, predicted, zero_division=0))

# Load dataset
data_clean = pd.read_csv('hasil_TextPreProcessing_edlink.csv')

SVM Accuracy: 0.8314606741573034
SVM Precision: 0.7910447761194029
SVM Recall: 0.9814814814814815
SVM f1_score: 0.8760330578512396
confusion_matrix:
 [[106   2]
 [ 28  42]]

              precision    recall  f1-score   support

     Negatif       0.79      0.98      0.88       108
     Positif       0.95      0.60      0.74        70

    accuracy                           0.83       178
   macro avg       0.87      0.79      0.81       178
weighted avg       0.86      0.83      0.82       178

